## **CLASSIFICATION of Diabetic Retinopathy using CUSTOM CNN and TRANSFER LEARNING**

![Static Badge](https://img.shields.io/badge/Made%20By-SHREENANDAN%20SAHU-5a189a?style=flat-square) ![Static Badge](https://img.shields.io/badge/Project%20Category-CNN-a53860?style=flat-square) 

![image.png](./results/thumbnail.png)






In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
# paths of different splited folders
train_dir = "drp/dataset/train"
val_dir = "drp/dataset/val"
test_dir = "drp/dataset/test"

# Data augmentation generating multiple copies to improve generality
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation & test data: only rescaling
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Train set
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Validation set
val_set = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Test set
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 2573 images belonging to 5 classes.
Found 549 images belonging to 5 classes.
Found 555 images belonging to 5 classes.


In [24]:
from tensorflow.keras.layers import GlobalMaxPooling2D

model = Sequential([
    Conv2D(64, (5, 5), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (5, 5), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (5, 5), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(1024, (1, 1), activation='relu'),
    MaxPooling2D(2, 2),
    

    GlobalMaxPooling2D(),  

    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(train_set.num_classes, activation='softmax')
])


In [25]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_set,
    validation_data=val_set,
    epochs=10
)

C:\Users\Shreenandan Sahu\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.5340 - loss: 1.2115 - val_accuracy: 0.7031 - val_loss: 0.9623
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.6630 - loss: 0.9845 - val_accuracy: 0.6539 - val_loss: 0.9073
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.6840 - loss: 0.9202 - val_accuracy: 0.7177 - val_loss: 0.8384
Epoch 4/10
37/81 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.6765 - loss: 0.9415

In [ ]:
from tensorflow.keras.models import Model
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Build model input/output if not built yet
model.build(input_shape=(None, 224, 224, 3))

# Create feature extractor from Dense(1024) layer before dropout
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)

def extract_features_labels(dataset, feature_model):
    features = []
    labels = []
    for i in range(len(dataset)):
        x_batch, y_batch = dataset[i]
        feats = feature_model.predict(x_batch)
        features.append(feats)
        labels.append(y_batch)
    features = np.vstack(features)
    labels = np.vstack(labels)
    return features, labels

# Extract features
train_features, train_labels = extract_features_labels(train_set, feature_extractor)
val_features, val_labels = extract_features_labels(val_set, feature_extractor)

# Convert one-hot labels to class indices
train_labels_int = np.argmax(train_labels, axis=1)
val_labels_int = np.argmax(val_labels, axis=1)

# Train SVM
svm_clf = SVC(kernel='rbf', gamma='scale')
svm_clf.fit(train_features, train_labels_int)
val_preds_svm = svm_clf.predict(val_features)
print("SVM Validation Accuracy:", accuracy_score(val_labels_int, val_preds_svm))

# Confusion matrix SVM
cm_svm = confusion_matrix(val_labels_int, val_preds_svm)
disp_svm = ConfusionMatrixDisplay(confusion_matrix=cm_svm, display_labels=list(train_set.class_indices.keys()))
disp_svm.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - SVM")
plt.show()

# Train Random Forest
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(train_features, train_labels_int)
val_preds_rf = rf_clf.predict(val_features)
print("Random Forest Validation Accuracy:", accuracy_score(val_labels_int, val_preds_rf))

# Confusion matrix RF
cm_rf = confusion_matrix(val_labels_int, val_preds_rf)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=list(train_set.class_indices.keys()))
disp_rf.plot(cmap=plt.cm.Greens)
plt.title("Confusion Matrix - Random Forest")
plt.show()


AttributeError: The layer sequential_2 has never been called and thus has no defined input.